In [28]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [5]:
from time import sleep

import twitter, re, datetime, pandas as pd
import sys
import re
import time
import tweepy
from pprint import pprint 

import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle 

In [2]:
# Refit model here

In [11]:
df_big=pd.concat([
    pd.read_csv('./data/5_cities_2019.1.3.csv'), 
    pd.read_csv('./data/5_cities_2019.1.17.csv')
])
df_big.index = range(df_big.shape[0])
df_big["tweet_time"] = pd.to_datetime(df_big.tweet_time)

In [12]:
df_big.groupby(['tweet_place', df_big.tweet_time.dt.date]).count().loc[:, ["tweet_text"]]

tweet_text
tweet_place       tweet_time            
Austin, TX        2019-01-03         358
                  2019-01-17         788
Boston, MA        2019-01-03          82
                  2019-01-17         625
Portland, OR      2019-01-03         442
                  2019-01-17         654
San Francisco, CA 2019-01-03         316
                  2019-01-17         654
Shreveport, LA    2019-01-03         455
                  2019-01-17         474

In [13]:
cvec = CountVectorizer()

df_big=df_big.join(pd.DataFrame(cvec.fit_transform(df_big.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names()))

In [14]:
df_big["tweet_time"] = pd.to_datetime(df_big.tweet_time)
df_big['is_rain']= (((df_big.tweet_place == "Shreveport, LA") & (df_big.tweet_time.dt.date == datetime.date(2019, 1, 3))) |\
                    ((df_big.tweet_place == "San Francisco, CA") & (df_big.tweet_time.dt.date == datetime.date(2019, 1, 17))))*1

In [15]:
excludes = ['tweet_text', 'tweet_time', 'tweet_place', 'is_rain', 'predicted']

In [16]:
X = df_big.loc[:, [col for col in df_big.columns if col not in excludes]]
y = df_big.is_rain

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

df_big['predicted'] = lr.predict(X)
df_big['probas'] = [element[1] for element in lr.predict_proba(X)]

df_big.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7805280528052805


,is_rain,predicted,predicted,probas
tweet_place,,,,
"Austin, TX",0.000000,0.011344,0.011344,0.131639
"Boston, MA",0.000000,0.008487,0.008487,0.134202
"Portland, OR",0.000000,0.006387,0.006387,0.116531
"San Francisco, CA",0.674227,0.451546,0.451546,0.428521
"Shreveport, LA",0.489774,0.274489,0.274489,0.327072


In [12]:
df_big[(df_big.tweet_place=="Austin, TX")&(df_big.tweet_time.dt.date>datetime.date(2019,1,4))].loc[:, 'probas'].mean()

0.12625038624702614

In [18]:
# Doesn't seem to help too much filter 
df_big[(df_big.tweet_place=="Austin, TX")&(df_big.tweet_time.dt.date>=datetime.date(2019,1,17))]\
.loc[:, ['tweet_time','probas']].sort_values(by='tweet_time').tail(20).mean()

probas    0.13503
dtype: float64

In [20]:
# pickle.dump(cvec, open('vectorizer.sav', 'wb')) 
# pickle.dump(lr, open('lin_regressor.sav', 'wb')) 

In [17]:
cvec = pickle.load(open('vectorizer.sav', 'rb')) 
lr = pickle.load(open('lin_regressor.sav', 'rb')) 

In [52]:
# check small frame
df_small = pd.read_csv('./data/austin_2019-01-27_to_2019-01-28.csv')
df_small["tweet_time"] = pd.to_datetime(df_small.tweet_time)

In [60]:
for i in range(1,10):
    print('{} to {} hours ago had {} tweets'.format(i , i+1, 
                                                   df_small[(df_small.tweet_time >= (datetime.datetime.now() - datetime.timedelta(hours=i+1)))&\
                                                          ( df_small.tweet_time <= (datetime.datetime.now() - datetime.timedelta(hours=i)))].shape[0]))

1 to 2 hours ago had 363 tweets
2 to 3 hours ago had 291 tweets
3 to 4 hours ago had 190 tweets
4 to 5 hours ago had 120 tweets
5 to 6 hours ago had 57 tweets
6 to 7 hours ago had 53 tweets
7 to 8 hours ago had 77 tweets
8 to 9 hours ago had 81 tweets
9 to 10 hours ago had 67 tweets


In [51]:
df_small = df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=4))] # Expand here to 6 if shape[0], then allow all
df_small = pd.concat([df_small, pd.DataFrame(cvec.transform(df_small.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names())], axis=1)
_X = df_small.loc[:, [col for col in df_small.columns if col in X.columns]]

_X.fillna(0, inplace=True)

df_small['predicted'] = lr.predict(_X)
df_small['probas'] = [element[1] for element in lr.predict_proba(_X)]

df_small.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,is_rain,predicted,predicted,probas
tweet_place,,,,
"Austin, TX",NaN,0.033199,0.033199,0.195745


# Script Needs

In [2]:
cvec = pickle.load(open('vectorizer.sav', 'rb')) 
lr = pickle.load(open('lin_regressor.sav', 'rb')) 

In [22]:
# One time
with open('./twitterapi.txt') as f:
    ck, cs, atk, ats = f.read().split(',')

In [23]:
# your keys go here:
twitter_keys = {
    'consumer_key':        ck,
    'consumer_secret':     cs,
    'access_token_key':    atk,
    'access_token_secret': ats
}

In [24]:
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [6]:
# update_interval_hours = 6
# for i in range(24):
#     sleep(update_interval_hours*60*60)

In [29]:
tomorrow = datetime.datetime.now() + datetime.timedelta(1)
yesterday = datetime.datetime.now() - datetime.timedelta(1)


In [32]:
# Each Time
end_date = '{}-{}-{}'.format(tomorrow.year, tomorrow.month, tomorrow.day)   
start_date = '{}-{}-{}'.format(yesterday.year, yesterday.month, yesterday.day)

In [33]:
# Otherwise, won't update public doc for 1 hour
# !gsutil -h "Cache-Control:no-cache, max-age=0" cp -a public-read myfile.json gs://mybucket

In [46]:
create_big = False
if create_big:
    df_big = pd.DataFrame()
    places = ["Shreveport, LA",'San Francisco, CA', "Austin, TX", "Portland, OR", "Boston, MA"]
else:
    df_small = pd.DataFrame()
    places = ["Austin, TX"]
tweetsPerQry = 100
counter = 0

for place_q in places: 

    place = api.geo_search(query=place_q, granularity="city")
    place_id = place[0].id

    max_id = -1
    for _ in range(20):
        try:
            if (max_id <= 0):
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, since=start_date, until=end_date)
            else:
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, max_id=str(max_id - 1), since=start_date, until=end_date)

            df_text = pd.DataFrame([new_tweets[i]._json['text'] for i in range(len(new_tweets))], columns=['tweet_text'])
            df_text['tweet_time']=[new_tweets[i]._json['created_at'] for i in range(len(new_tweets))]
            df_text['tweet_place'] = place_q
            if create_big:
                df_big = pd.concat([df_big, df_text])
            else:
                df_small = pd.concat([df_small, df_text])
            
            if not new_tweets:
                counter += 1
                if create_big:
                    df_big.to_csv('./collected_tweets_{}.csv'.format(counter)) 
                else: 
                    df_small.to_csv('./collected_tweets_{}.csv'.format(counter)) # Should provide log of last good pull at least
                    
                print("No more tweets found")
                break

            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            print('    all_done')
            break
        sleep(5)

In [50]:
df_small.to_csv('./data/austin_2019-01-27_to_2019-01-28.csv', index=False)

In [ ]:
# df_big.to_csv('./5_cities_2019.1.17.csv', index=False)

In [11]:
model_columns = cvec.get_feature_names()
df_small = pd.concat([df_small.reset_index().drop('index', axis=1), pd.DataFrame(cvec.transform(df_small.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names())], axis=1)
_X = df_small.loc[:, [col for col in df_small.columns if col in model_columns]]

_X.fillna(0, inplace=True)

df_small['predicted'] = lr.predict(_X)
df_small['probas'] = [element[1] for element in lr.predict_proba(_X)]

In [12]:
excludes = ['tweet_text', 'tweet_time', 'tweet_place', 'is_rain', 'predicted']
_rain_probability = df_small.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()['probas'].values[0]
_rain_probability = round(_rain_probability*100, 1)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
